In [ ]:
import pandas as pd 
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import rcParams
import re

import pandas as pd 
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import rcParams
import re
from sklearn.preprocessing import StandardScaler

%matplotlib inline

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LinearRegression

from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
import keras
from tensorflow.keras.optimizers import Adam
from keras.optimizers import SGD
import graphviz
!pip3 install ann_visualizer
from ann_visualizer.visualize import ann_viz

from sklearn.metrics import r2_score
from sklearn.metrics import roc_curve
from sklearn.metrics import auc
from sklearn.metrics import precision_recall_curve
from sklearn import metrics


rcParams['figure.figsize'] = 10,8


%matplotlib inline

rcParams['figure.figsize'] = 10,8

In [ ]:
df_train = pd.read_csv("train.csv")
df_test = pd.read_csv("test.csv")

In [ ]:
print(df_train.info())
print(df_test.info())

In [ ]:
df_train.describe()

In [ ]:
print(df_train.head())

In [ ]:
df_train['Title'] = df_train.Name.apply(lambda x: re.search(' ([A-Z][a-z]+)\.', x).group(1))

plt.figure(figsize=(12,5))

sns.countplot(x='Title', data=df_train, palette="hls")
plt.xlabel("Title", fontsize=16) #seting the xtitle and size
plt.ylabel("Count", fontsize=16) # Seting the ytitle and size
plt.title("Title Name Count", fontsize=20) 
plt.xticks(rotation=45)
plt.show()

In [ ]:
df_test['Title'] = df_test.Name.apply(lambda x: re.search(' ([A-Z][a-z]+)\.', x).group(1))

In [ ]:
Title_Dictionary = {
        "Capt":       "Officer",
        "Col":        "Officer",
        "Major":      "Officer",
        "Dr":         "Officer",
        "Rev":        "Officer",
        "Jonkheer":   "Royalty",
        "Don":        "Royalty",
        "Sir" :       "Royalty",
        "the Countess":"Royalty",
        "Dona":       "Royalty",
        "Lady" :      "Royalty",
        "Mme":        "Mrs",
        "Ms":         "Mrs",
        "Mrs" :       "Mrs",
        "Mlle":       "Miss",
        "Miss" :      "Miss",
        "Mr" :        "Mr",
        "Master" :    "Master"
                   }
    
df_train['Title'] = df_train.Title.map(Title_Dictionary)
df_test['Title'] = df_test.Title.map(Title_Dictionary)

In [ ]:
print("Chances to survive based on titles: ") 
print(df_train.groupby("Title")["Survived"].mean())

plt.figure(figsize=(12,5))

sns.countplot(x='Title', data=df_train, palette="hls",
              hue="Survived")
plt.xlabel("Titles", fontsize=16)
plt.ylabel("Count", fontsize=16)
plt.title("Title Grouped Count", fontsize=20)
plt.xticks(rotation=45)
plt.show()

In [ ]:
age_high_zero_died = df_train[(df_train["Age"] > 0) & 
                              (df_train["Survived"] == 0)]
age_high_zero_surv = df_train[(df_train["Age"] > 0) & 
                              (df_train["Survived"] == 1)]

plt.figure(figsize=(10,5))

sns.distplot(age_high_zero_surv["Age"], bins=24, color='g')
sns.distplot(age_high_zero_died["Age"], bins=24, color='r')
plt.title("Distribuition and density by Age",fontsize=20)
plt.xlabel("Age",fontsize=15)
plt.ylabel("Distribuition Died and Survived",fontsize=15)
plt.show()

In [ ]:
age_group = df_train.groupby(["Sex","Pclass","Title"])["Age"]

print(age_group.median())

In [ ]:
df_train.loc[df_train.Age.isnull(), 'Age'] = df_train.groupby(['Sex','Pclass','Title']).Age.transform('median')

In [ ]:
plt.figure(figsize=(12,5))

sns.distplot(df_train["Age"], bins=24)
plt.title("Distribuition and density by Age")
plt.xlabel("Age")
plt.show()

In [ ]:
plt.figure(figsize=(12,5))

g = sns.FacetGrid(df_train, col='Survived',size=5)
g = g.map(sns.distplot, "Age")
plt.show()

In [ ]:
interval = (0, 5, 12, 18, 25, 35, 60, 120) 

cats = ['babies', 'Children', 'Teen', 'Student', 'Young', 'Adult', 'Senior']

df_train["Age_cat"] = pd.cut(df_train.Age, interval, labels=cats)

df_train["Age_cat"].head()

In [ ]:
interval = (0, 5, 12, 18, 25, 35, 60, 120)

cats = ['babies', 'Children', 'Teen', 'Student', 'Young', 'Adult', 'Senior']

df_test["Age_cat"] = pd.cut(df_test.Age, interval, labels=cats)

In [ ]:
print(pd.crosstab(df_train.Age_cat, df_train.Survived))

plt.figure(figsize=(12,10))

plt.subplot(2,1,1)
sns.countplot("Age_cat",data=df_train,hue="Survived", palette="hls")
plt.ylabel("Count", fontsize=18)
plt.xlabel("Age Categories", fontsize=18)
plt.title("Age Distribution ", fontsize=20)

plt.subplot(2,1,2)
sns.swarmplot(x='Age_cat',y="Fare",data=df_train,
              hue="Survived", palette="hls", )
plt.ylabel("Fare Distribution", fontsize=18)
plt.xlabel("Age Categories", fontsize=18)
plt.title("Fare Distribution by Age Categories ", fontsize=20)

plt.subplots_adjust(hspace = 0.5, top = 0.9)

plt.show()

In [ ]:
Age_fare = ['Pclass', 'Age_cat']

cm = sns.light_palette("green", as_cmap=True)
pd.crosstab(df_train[Age_fare[0]], df_train[Age_fare[1]], 
            values=df_train['Fare'], aggfunc=['mean']).style.background_gradient(cmap = cm)


In [ ]:
plt.figure(figsize=(12,5))

sns.distplot(df_train[df_train.Survived == 0]["Fare"], 
             bins=50, color='r')
sns.distplot(df_train[df_train.Survived == 1]["Fare"], 
             bins=50, color='g')
plt.title("Fare Distribuition by Survived", fontsize=20)
plt.xlabel("Fare", fontsize=15)
plt.ylabel("Density",fontsize=15)
plt.show()

In [ ]:
df_train.Fare = df_train.Fare.fillna(-0.5)

quant = (-1, 0, 8, 15, 31, 600)

label_quants = ['NoInf', 'quart_1', 'quart_2', 'quart_3', 'quart_4']

df_train["Fare_cat"] = pd.cut(df_train.Fare, quant, labels=label_quants)

print(pd.crosstab(df_train.Fare_cat, df_train.Survived))

plt.figure(figsize=(12,5))

sns.countplot(x="Fare_cat", hue="Survived", data=df_train, palette="hls")
plt.title("Count of survived x Fare expending",fontsize=20)
plt.xlabel("Fare Cat",fontsize=15)
plt.ylabel("Count",fontsize=15)

plt.show()

In [ ]:
df_test.Fare = df_test.Fare.fillna(-0.5)

quant = (-1, 0, 8, 15, 31, 1000)
label_quants = ['NoInf', 'quart_1', 'quart_2', 'quart_3', 'quart_4']

df_test["Fare_cat"] = pd.cut(df_test.Fare, quant, labels=label_quants)

In [ ]:
del df_train["Fare"]
del df_train["Ticket"]
del df_train["Age"]
del df_train["Cabin"]
del df_train["Name"]

del df_test["Fare"]
del df_test["Ticket"]
del df_test["Age"]
del df_test["Cabin"]
del df_test["Name"]

In [ ]:
print(pd.crosstab(df_train.Pclass, df_train.Embarked))

plt.figure(figsize=(12,5))

sns.countplot(x="Embarked", data=df_train, hue="Pclass",palette="hls")
plt.title('Embarked x Pclass Count', fontsize=20)
plt.xlabel('Embarked with PClass',fontsize=17)
plt.ylabel('Count', fontsize=17)

plt.show()

In [ ]:
df_train["Embarked"] = df_train["Embarked"].fillna('S')

print(pd.crosstab(df_train.Survived, df_train.Embarked))

plt.figure(figsize=(12,5))

sns.countplot(x="Embarked", data=df_train, hue="Survived",palette="hls")
plt.title('Class Distribuition by survived or not',fontsize=20)
plt.xlabel('Embarked',fontsize=17)
plt.ylabel('Count', fontsize=17)

plt.show()

print(pd.crosstab(df_train.Survived, df_train.Pclass))

plt.figure(figsize=(12,5))

sns.countplot(x="Pclass", data=df_train, hue="Survived",palette="hls")
plt.xlabel('PClass',fontsize=17)
plt.ylabel('Count', fontsize=17)
plt.title('Class Distribuition by Survived or not', fontsize=20)

plt.show()

In [ ]:
g = sns.factorplot(x="SibSp",y="Survived",data=df_train,
                   kind="bar", height = 5, aspect= 1.6, palette = "hls")
g.set_ylabels("Survival Probability", fontsize=15)
g.set_xlabels("SibSp", fontsize=15)

plt.show()

g  = sns.factorplot(x="Parch",y="Survived",data=df_train, kind="bar", size = 6,palette = "hls")

g = g.set_ylabels("Survival Probability")
g.set_xlabels("Parch", fontsize=15)

plt.show()

In [ ]:
df_train["FSize"] = df_train["Parch"] + df_train["SibSp"] + 1

df_test["FSize"] = df_test["Parch"] + df_test["SibSp"] + 1

print(pd.crosstab(df_train.FSize, df_train.Survived))
sns.factorplot(x="FSize",y="Survived", data=df_train, kind="bar",size=6, aspect=1.6)
plt.show()

del df_train["SibSp"]
del df_train["Parch"]

del df_test["SibSp"]
del df_test["Parch"]

In [ ]:
print(pd.crosstab(df_train.Survived, df_train.Sex))

plt.figure(figsize=(12,5))
sns.countplot(x="Sex", data=df_train, hue="Survived",palette="hls")
plt.title('Sex Distribution by Survived or not', fontsize=20)
plt.xlabel('Sex Distribution',fontsize=17)
plt.ylabel('Count', fontsize=17)

plt.show()

In [ ]:
print("Total of Survived or not: ")
print(df_train.groupby("Survived")["PassengerId"].count())

plt.figure(figsize=(12,5))

sns.countplot(x="Survived", data=df_train,palette="hls")
plt.title('Total Distribuition by survived or not', fontsize=22)
plt.xlabel('Target Distribuition', fontsize=18)
plt.ylabel('Count', fontsize=18)

plt.show()

In [ ]:
df_train.head()

In [ ]:
df_train = pd.get_dummies(df_train, columns=["Sex","Embarked","Age_cat","Fare_cat","Title"],\
                          prefix=["Sex","Emb","Age","Fare","Prefix"], drop_first=True)

df_test = pd.get_dummies(df_test, columns=["Sex","Embarked","Age_cat","Fare_cat","Title"],\
                         prefix=["Sex","Emb","Age","Fare","Prefix"], drop_first=True)

plt.figure(figsize=(15,12))
plt.title('Correlation of Features for Train Set')
sns.heatmap(df_train.astype(float).corr(),vmax=1.0,  annot=True)
plt.show()

In [ ]:
train = df_train.drop(["Survived","PassengerId"],axis=1)
train_ = df_train["Survived"]

test_ = df_test.drop(["PassengerId"],axis=1)

X_train = train.values
y_train = train_.values

X_test = test_.values
X_test = X_test.astype(np.float64, copy=False)

sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.fit_transform(X_test)

df_submission = pd.read_csv("gender_submission.csv")
submission_ = df_submission["Survived"]
Y_test = submission_.values

In [ ]:
# Decision Tree
decision_tree = DecisionTreeClassifier()
decision_tree.fit(X_train, y_train)
Y_pred = decision_tree.predict(X_test)
acc_decision_tree = round(decision_tree.score(X_train, y_train) * 100, 2)

In [ ]:
# Gaussian Naive Bayes
gaussian = GaussianNB()
gaussian.fit(X_train, y_train)
Y_pred = gaussian.predict(X_test)
acc_gaussian = round(gaussian.score(X_train, y_train) * 100, 2)

In [ ]:
# Logistic Regression
logreg = LogisticRegression()
logreg.fit(X_train, y_train)
Y_pred = logreg.predict(X_test)
acc_log = round(logreg.score(X_train, y_train) * 100, 2)

In [ ]:
# KNN
knn = KNeighborsClassifier(n_neighbors = 3)
knn.fit(X_train, y_train)
Y_pred = knn.predict(X_test)
acc_knn = round(knn.score(X_train, y_train) * 100, 2)

In [ ]:
# Support Vector Machines
svc = SVC(probability=True)
svc.fit(X_train, y_train)
Y_pred = svc.predict(X_test)
acc_svc = round(svc.score(X_train, y_train) * 100, 2)

In [ ]:
# Perceptron
perceptron = Perceptron()
perceptron.fit(X_train, y_train)
Y_pred_perceptron = perceptron.predict(X_test)
acc_perceptron = round(perceptron.score(X_train, y_train) * 100, 2)

In [ ]:
#Neural Network
#Neural Network Structure
model = Sequential()

model.add(Dense(18, 
                activation='relu',  
                input_dim=20,
                kernel_initializer='uniform'))

model.add(Dropout(0.2))

model.add(Dense(60,
                kernel_initializer='uniform',
                activation='relu'))

model.add(Dropout(0.7))

model.add(Dense(1,
                kernel_initializer='uniform',
                activation='sigmoid'))

model.summary()

#Neural Network Compile & Fit
model.compile(optimizer = Adam(learning_rate=0.001), 
#model22.compile(optimizer = SGD(lr = 0.01, momentum = 0.9),    
                   loss = 'binary_crossentropy', 
                   metrics = ['accuracy'])

pred_NN = model.fit(X_train, y_train, validation_data=(X_test,Y_test),
               batch_size = 32, 
               epochs = 200, verbose=1, use_multiprocessing=True)

#Neural Network Score
scores = model.evaluate(X_train, y_train, batch_size=25)
acc_NN = round(scores[1]*100, 2)

In [ ]:
ann_viz(model, title="Neural Network")

In [ ]:
fig, ax_arr = plt.subplots(nrows = 4, ncols = 2, figsize = (15,20))

#DT
probs = decision_tree.predict_proba(X_test)
preds_tree = probs[:,1]
fprdtree, tprdtree, thresholddtree = roc_curve(Y_test, preds_tree)
roc_aucdtree = auc(fprdtree, tprdtree)

ax_arr[0,0].plot(fprdtree, tprdtree, 'b', label = 'AUC = %0.2f' % roc_aucdtree)
ax_arr[0,0].plot([0, 1], [0, 1],'r--')
ax_arr[0,0].set_title('ROC Decision Tree ',fontsize=20)
ax_arr[0,0].set_ylabel('True Positive Rate',fontsize=20)
ax_arr[0,0].set_xlabel('False Positive Rate',fontsize=15)
ax_arr[0,0].legend(loc = 'lower right', prop={'size': 16})

#NB
probs = gaussian.predict_proba(X_test)
preds_NB = probs[:,1]
fprNB, tprNB, thresholdNB = roc_curve(Y_test, preds_NB)
roc_aucNB = auc(fprNB, tprNB)

ax_arr[0,1].plot(fprNB, tprNB, 'b', label = 'AUC = %0.2f' % roc_aucNB)
ax_arr[0,1].plot([0, 1], [0, 1],'r--')
ax_arr[0,1].set_title('ROC Gaussian Naive Bayes ',fontsize=20)
ax_arr[0,1].set_ylabel('True Positive Rate',fontsize=20)
ax_arr[0,1].set_xlabel('False Positive Rate',fontsize=15)
ax_arr[0,1].legend(loc = 'lower right', prop={'size': 16})

#LReg
probs = logreg.predict_proba(X_test)
preds_logreg = probs[:,1]
fprlogreg, tprlogreg, thresholdlogreg = roc_curve(Y_test, preds_logreg)
roc_auclogreg = auc(fprlogreg, tprlogreg)

ax_arr[1,0].plot(fprlogreg, tprlogreg, 'b', label = 'AUC = %0.2f' % roc_auclogreg)
ax_arr[1,0].plot([0, 1], [0, 1],'r--')
ax_arr[1,0].set_title('ROC Logistic Regression ',fontsize=20)
ax_arr[1,0].set_ylabel('True Positive Rate',fontsize=20)
ax_arr[1,0].set_xlabel('False Positive Rate',fontsize=15)
ax_arr[1,0].legend(loc = 'lower right', prop={'size': 16})

#KNN
probs = knn.predict_proba(X_test)
preds_KNN = probs[:,1]
fprkNN, tprkNN, thresholdkNN = roc_curve(Y_test, preds_KNN)
roc_auckNN = auc(fprkNN, tprkNN)

ax_arr[1,1].plot(fprkNN, tprkNN, 'b', label = 'AUC = %0.2f' % roc_auckNN)
ax_arr[1,1].plot([0, 1], [0, 1],'r--')
ax_arr[1,1].set_title('ROC k-Nearest Neighbors ',fontsize=20)
ax_arr[1,1].set_ylabel('True Positive Rate',fontsize=20)
ax_arr[1,1].set_xlabel('False Positive Rate',fontsize=15)
ax_arr[1,1].legend(loc = 'lower right', prop={'size': 16})

#SVC
probs = svc.predict_proba(X_test)
preds_SVC = probs[:,1]
fprsvc, tprsvc, thresholdsvc = roc_curve(Y_test, preds_SVC)
roc_aucsvc = auc(fprsvc, tprsvc)

ax_arr[2,0].plot(fprsvc, tprsvc, 'b', label = 'AUC = %0.2f' % roc_aucsvc)
ax_arr[2,0].plot([0, 1], [0, 1],'r--')
ax_arr[2,0].set_title('ROC Support Vector Machine',fontsize=20)
ax_arr[2,0].set_ylabel('True Positive Rate',fontsize=20)
ax_arr[2,0].set_xlabel('False Positive Rate',fontsize=15)
ax_arr[2,0].legend(loc = 'lower right', prop={'size': 16})


#Perceptron
probs = perceptron.decision_function(X_test)
preds_prtn = probs
fprprtn, tprprtn, thresholdprtn = roc_curve(Y_test, preds_prtn)
roc_aucprtn = auc(fprprtn, tprprtn)

ax_arr[2,1].plot(fprprtn, tprprtn, 'b', label = 'AUC = %0.2f' % roc_aucprtn)
ax_arr[2,1].plot([0, 1], [0, 1],'r--')
ax_arr[2,1].set_title('ROC Perceptron ',fontsize=20)
ax_arr[2,1].set_ylabel('True Positive Rate',fontsize=20)
ax_arr[2,1].set_xlabel('False Positive Rate',fontsize=15)
ax_arr[2,1].legend(loc = 'lower right', prop={'size': 16})


#NN
probs = model.predict_proba(X_test)
preds_NN = probs
fprNN, tprNN, thresholdNN = roc_curve(Y_test, preds_NN)
roc_aucNN = auc(fprNN, tprNN)

ax_arr[3,0].plot(fprNN, tprNN, 'b', label = 'AUC = %0.2f' % roc_aucNN)
ax_arr[3,0].plot([0, 1], [0, 1],'r--')
ax_arr[3,0].set_title('ROC Neural Network ',fontsize=20)
ax_arr[3,0].set_ylabel('True Positive Rate',fontsize=20)
ax_arr[3,0].set_xlabel('False Positive Rate',fontsize=15)
ax_arr[3,0].legend(loc = 'lower right', prop={'size': 16})

#ALL
ax_arr[3,1].plot(fprdtree, tprdtree, 'b', label = 'Decision Tree', color='black')
ax_arr[3,1].plot(fprNB, tprNB, 'b', label = 'Gaussian Naive Bayes', color='blue')
ax_arr[3,1].plot(fprlogreg, tprlogreg, 'b', label = 'Logistic Regression', color='brown')
ax_arr[3,1].plot(fprkNN, tprkNN, 'b', label = 'k-Nearest Neighbors', color='green')
ax_arr[3,1].plot(fprsvc, tprsvc, 'b', label = 'Support Vector Machine', color='grey')
ax_arr[3,1].plot(fprprtn, tprprtn, 'b', label = 'Perceptron', color='red')
ax_arr[3,1].plot(fprNN, tprNN, 'b', label = 'Neural Network', color='orange')
ax_arr[3,1].set_title('ROC ',fontsize=20)
ax_arr[3,1].set_ylabel('True Positive Rate',fontsize=20)
ax_arr[3,1].set_xlabel('False Positive Rate',fontsize=15)
ax_arr[3,1].legend(loc = 'lower right', prop={'size': 16})

plt.subplots_adjust(wspace=0.2)
plt.tight_layout() 

In [ ]:
fig, ax_arr = plt.subplots(nrows = 4, ncols = 2, figsize = (15,20))

#DT
prdtree, redtree, thsld_tree = precision_recall_curve(Y_test, preds_tree)
pr_aucdtree = auc(redtree, prdtree)

ax_arr[0,0].plot(prdtree, redtree, 'b', label = 'AUC = %0.2f' % pr_aucdtree)
ax_arr[0,0].set_title('PR Decision ',fontsize=20)
ax_arr[0,0].set_ylabel('Precision',fontsize=20)
ax_arr[0,0].set_xlabel('Recall',fontsize=15)
ax_arr[0,0].legend(loc = 'lower left', prop={'size': 16})

#NB
prNB, reNB, thsld_NB = precision_recall_curve(Y_test, preds_NB)
pr_aucNB = auc(reNB, prNB)

ax_arr[0,1].plot(prNB, reNB, 'b', label = 'AUC = %0.2f' % pr_aucNB)
ax_arr[0,1].set_title('PR Gaussian Naive Bayes ',fontsize=20)
ax_arr[0,1].set_ylabel('Precision',fontsize=20)
ax_arr[0,1].set_xlabel('Recall',fontsize=15)
ax_arr[0,1].legend(loc = 'lower left', prop={'size': 16})

#LReg
prlogreg, relogreg, thsld_logreg = precision_recall_curve(Y_test, preds_logreg)
pr_auclogreg = auc(relogreg, prlogreg)

ax_arr[1,0].plot(prlogreg, relogreg, 'b', label = 'AUC = %0.2f' % pr_auclogreg)
ax_arr[1,0].set_title('PR Logistic Regression ',fontsize=20)
ax_arr[1,0].set_ylabel('Precision',fontsize=20)
ax_arr[1,0].set_xlabel('Recall',fontsize=15)
ax_arr[1,0].legend(loc = 'lower left', prop={'size': 16})

#KNN
prkNN, rekNN, thsld_kNN = precision_recall_curve(Y_test, preds_KNN)
pr_auckNN = auc(rekNN, prkNN)

ax_arr[1,1].plot(prkNN, rekNN, 'b', label = 'AUC = %0.2f' % pr_auckNN)
ax_arr[1,1].set_title('PR k-Nearest Neighbors ',fontsize=20)
ax_arr[1,1].set_ylabel('Precision',fontsize=20)
ax_arr[1,1].set_xlabel('Recall',fontsize=15)
ax_arr[1,1].legend(loc = 'lower left', prop={'size': 16})

#SVC
prsvc, resvc, thsld_SVC = precision_recall_curve(Y_test, preds_SVC)
pr_aucSVC = auc(resvc, prsvc)

ax_arr[2,0].plot(prsvc, resvc, 'b', label = 'AUC = %0.2f' % pr_aucSVC)
ax_arr[2,0].set_title('PR Support Vector Machine',fontsize=20)
ax_arr[2,0].set_ylabel('Precision',fontsize=20)
ax_arr[2,0].set_xlabel('Recall',fontsize=15)
ax_arr[2,0].legend(loc = 'lower left', prop={'size': 16})


#Perceptron
prprtn, reprtn, thsld_prtn = precision_recall_curve(Y_test, preds_prtn)
pr_aucprtn = auc(reprtn, prprtn)

ax_arr[2,1].plot(prprtn, reprtn, 'b', label = 'AUC = %0.2f' % pr_aucprtn)
ax_arr[2,1].set_title('PR Perceptron ',fontsize=20)
ax_arr[2,1].set_ylabel('Precision',fontsize=20)
ax_arr[2,1].set_xlabel('Recall',fontsize=15)
ax_arr[2,1].legend(loc = 'lower left', prop={'size': 16})


#NN
prNN, reNN, thsld_NN = precision_recall_curve(Y_test, preds_NN)
pr_aucNN = auc(reNN, prNN)

ax_arr[3,0].plot(prNN, reNN, 'b', label = 'AUC = %0.2f' % pr_aucNN)
ax_arr[3,0].set_title('PR Neural Network ',fontsize=20)
ax_arr[3,0].set_ylabel('Precision',fontsize=20)
ax_arr[3,0].set_xlabel('Recall',fontsize=15)
ax_arr[3,0].legend(loc = 'lower left', prop={'size': 16})

#ALL
ax_arr[3,1].plot(prdtree, redtree, 'b', label = 'Decision Tree', color='black')
ax_arr[3,1].plot(prNB, reNB, 'b', label = 'Gaussian Naive Bayes', color='blue')
ax_arr[3,1].plot(prlogreg, relogreg, 'b', label = 'Logistic Regression', color='brown')
ax_arr[3,1].plot(prkNN, rekNN, 'b', label = 'k-Nearest Neighbors', color='green')
ax_arr[3,1].plot(prsvc, resvc, 'b', label = 'Support Vector Machine', color='grey')
ax_arr[3,1].plot(prprtn, reprtn, 'b', label = 'Perceptron', color='red')
ax_arr[3,1].plot(prNN, reNN, 'b', label = 'Neural Network', color='orange')
ax_arr[3,1].set_title('PR ',fontsize=20)
ax_arr[3,1].set_ylabel('Precision',fontsize=20)
ax_arr[3,1].set_xlabel('Recall',fontsize=15)
ax_arr[3,1].legend(loc = 'lower left', prop={'size': 16})

plt.subplots_adjust(wspace=0.2)
plt.tight_layout() 

In [ ]:
plt.plot(pred_NN.history['accuracy'])
plt.plot(pred_NN.history['val_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()


In [ ]:
plt.plot(pred_NN.history['loss'])
plt.plot(pred_NN.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
models = pd.DataFrame({
    'Model': ['SVM', 'KNN', 'Logistic Regression', 
              'Naive Bayes', 'Perceptron', 'Decision Tree', 'Neural Network'],
    'Score by Accuracy %': [acc_svc, acc_knn, acc_log, 
              acc_gaussian, acc_perceptron, 
              acc_decision_tree, acc_NN]})
sorted_score = models.sort_values(by='Score by Accuracy %', ascending=False)
print(sorted_score)

plt.figure(figsize=(12,5))

g = sns.factorplot(x="Model",y="Score by Accuracy %",data=sorted_score,
                   kind="bar", height = 5, aspect= 1.8, palette = "hls", hue="Score by Accuracy %", dodge=False)
g.set_ylabels("Score by Accuracy %", fontsize=15)
g.set_xlabels("Model", fontsize=15)

plt.show()